In [1]:
import pandas as pd
import glob
import os

path_google = '../..'
path = path_google + r'/04DB/01suscriptora/' # use your path
li = [path + 'suscriptora_202201.parquet',path + 'suscriptora_202202.parquet',path + 'suscriptora_202203.parquet',path + 'suscriptora_202204.parquet']

df_suscri = pd.concat(map(pd.read_parquet, li))
df_adenda = pd.read_parquet(path_google + '/04DB/02adenda/adenda.parquet')
df_digita = pd.read_parquet(path_google + '/04DB/03digital/digital.parquet')
df_roamin = pd.read_parquet(path_google + '/04DB/04roaming/roaming.parquet')
df_termin = pd.read_parquet(path_google + '/04DB/05terminal/terminal.parquet')
df_trafic = pd.read_parquet(path_google + '/04DB/06trafico/trafico.parquet')
df_conver = pd.read_parquet(path_google + '/04DB/07convergente/convergente.parquet')
df_target_new = pd.read_parquet(path_google + '/04DB/08target/target.parquet')
df_target = pd.read_parquet(path_google + '/01Data/02parquet/08_target_202201_202203.parquet')
df_target.columns =['numperiodo', 'nro_telefono_hash', 'target']

In [2]:
df_model = pd.merge(left=df_suscri,right= df_adenda,on=['numperiodo','nro_telefono_hash'],how='left')
df_model = pd.merge(left=df_model,right= df_digita,on=['numperiodo','nro_telefono_hash'],how='left')
df_model = pd.merge(left=df_model,right= df_roamin,on=['numperiodo','nro_telefono_hash'],how='left')
df_model = pd.merge(left=df_model,right= df_termin,on=['numperiodo','nro_telefono_hash'],how='left')
df_model = pd.merge(left=df_model,right= df_trafic,on=['numperiodo','nro_telefono_hash'],how='left')
df_model = pd.merge(left=df_model,right= df_conver,on=['numperiodo','nro_documento_hash'],how='left')
df_model = pd.merge(left=df_model,right= df_target,on=['numperiodo','nro_telefono_hash'],how='left')
df_model.rename(columns={'numperiodo':'periodo'},inplace=True)
df_model = pd.merge(left=df_model,right= df_target_new,on=['periodo','nro_telefono_hash'],how='left')

## 01 Construcción de algoritmo que en el mes anterior renovaron sus equipos

In [3]:
def prepare_df(df):
    df.set_index(['nro_documento_hash','nro_telefono_hash'],inplace=True)
    df.drop(columns=['fecingresocliente','fecactivacioncontrato','fechatraficomin','fechatraficomax','minutosstd','mensajesmean','mensajesstd','lanzamiento','target_new'],inplace =True)
    #df['target'] = df['target'].astype('int')
    return df
df = prepare_df(df_model.query('target_new >=0').copy())

In [18]:
print(df.shape)
print('No renovación', round(df['target'].value_counts()[0]/len(df) * 100,2), '% del dataset')
print('Renovación', round(df['target'].value_counts()[1]/len(df) * 100,2), '% del dataset')

(10855, 55)
No renovación 51.27 % del dataset
Renovación 48.73 % del dataset


In [19]:
from sklearn.model_selection import train_test_split
seed = 2106
X = df.drop(columns=['target'])
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed, stratify=y_train)

In [20]:
print('X_train', X_train.shape ,'No renovación', round(y_train.value_counts()[0]/len(X_train) * 100,2), '% del dataset')
print('X_test', X_test.shape,'No renovación', round(y_test.value_counts()[0]/len(X_test) * 100,2), '% del dataset')
print('X_valid', X_valid.shape,'No renovación', round(y_valid.value_counts()[0]/len(X_valid) * 100,2), '% del dataset')

X_train (6947, 54) No renovación 51.26 % del dataset
X_test (2171, 54) No renovación 51.27 % del dataset
X_valid (1737, 54) No renovación 51.3 % del dataset


In [39]:
from optbinning import BinningProcess

ModuleNotFoundError: No module named 'optbinning'

In [26]:
import warnings
warnings.filterwarnings("ignore")

In [33]:
import category_encoders as ce

encoder_target = ce.TargetEncoder(cols=['tipo_adq'])
encoder_woe = ce.WOEEncoder(cols=['tiposervicio'])
encoder_summary = ce.SummaryEncoder(cols=['modelo'])

encoder_target.fit(X_train, y_train)
encoder_woe.fit(X_train, y_train)
encoder_summary.fit(X_train, y_train)
X_test_cleaned = encoder_target.transform(X_test)
X_test_cleaned = encoder_woe.transform(X_test_cleaned)
X_test_cleaned = encoder_summary.transform(X_test_cleaned)

In [38]:
X_test_cleaned.modelo_75.value_counts()

1.000000    1881
0.009346      45
0.833333      19
0.500000      19
0.033333      18
0.625000      18
0.021277      14
0.022222      14
0.012987      13
0.034483      12
0.250000      10
0.045455       9
0.037037       9
0.035714       9
0.038462       9
0.043478       9
0.562500       8
0.755319       8
0.200000       6
0.041667       6
0.028571       6
0.333333       5
0.083333       4
0.066667       4
0.166667       4
0.100000       3
0.125000       2
0.071429       2
0.772727       2
0.541667       2
0.400000       1
Name: modelo_75, dtype: int64

In [28]:
#exclude by high missing values
feature_missing = ['minutosmean','minutosmean','gigasmean', 'gigasstd', 'gigassum', 'mensajesmean', 'mensajesstd', 'mensajessum']

,,periodo,tipo_adq,antiguedad_cliente,antiguedad_contrato,antiguedad_cliente_contrato,vchmesadenda,vchpenalidad,con_adenda,grupo,scorecat,...,trafico_app_6,trafico_app_7,trafico_app_8,trafico_app_9,trafico_total,giro,subgiro,tiene_prod_1,tiene_prod_2,tiene_prod_3
nro_documento_hash,nro_telefono_hash,,,,,,,,,,,,,,,,,,,,,
8c2d07fd461e79e55279cbda87c18cf49b254a655d39e6d97a4db7ca053d6f63,7d9a90fe29e000018a2d24c50ae35daae9dbdbe13f8e224c64ca7dd4a5f98f17,202203,0.362856,69.0,36.0,33.0,NaN,NaN,NaN,grupo_7|grupo_8,bajo,...,0.0,0.000000,123.637921,0.00000,257.374208,5b8e595b8cebb006e39a3d289e465a5718996785008cbb...,bd1b475ded03bdb3ab78de2f74782ad855c3c4b085d167...,1,0,0
da8826043d815eb8ba0e7fa1453f17567429936ef5f9f7cbf36beceefdad2f5c,6ddca7fa2aa30885fa2740b102ebfa4d6d88f0248ff87ab9558e08ad039108f3,202203,0.362856,25.0,25.0,0.0,NaN,NaN,NaN,grupo_1|grupo_10|grupo_8,muy alto,...,0.0,2.624814,0.000000,0.00000,6257.065578,5b8e595b8cebb006e39a3d289e465a5718996785008cbb...,bd1b475ded03bdb3ab78de2f74782ad855c3c4b085d167...,0,0,0
2bec1edc19b68c9c88e8ef5d863a5df92b1fdcda89917fffbc69b7488c25c131,afb0e87594e532178d5f9ae907dff2c079574d65944c2f2c2edd49de58e646b9,202202,0.362856,40.0,29.0,11.0,NaN,NaN,NaN,grupo_6|grupo_10|grupo_8,medio,...,0.0,0.000000,0.000000,0.00000,2246.847301,5b8e595b8cebb006e39a3d289e465a5718996785008cbb...,bd1b475ded03bdb3ab78de2f74782ad855c3c4b085d167...,0,0,0
aa31dc9670191783255595fe459ce7bb83d572d6cee3d9890c2747ecc46ee0eb,a6c46991d4c018ddedae3918e30c679e9328c2e89fb618ff955769fafb59c5e3,202202,0.362856,35.0,35.0,0.0,NaN,NaN,NaN,grupo_1|grupo_6|grupo_8,muy alto,...,0.0,0.000000,0.000000,0.00000,1454.219055,5b8e595b8cebb006e39a3d289e465a5718996785008cbb...,bd1b475ded03bdb3ab78de2f74782ad855c3c4b085d167...,0,0,0
7d66c1b6f2ea4bbde1a5a7d5babe4a5e24951a9d2305ef13c678ff3a319eeb21,e813cb9016e7bc86af63e70cb87ba1c4ed659c69aa4d6dec6426dddd1cd9ab7e,202202,0.362856,5.0,5.0,0.0,NaN,NaN,NaN,grupo_1|grupo_7|grupo_8|grupo_4,medio,...,0.0,0.000000,0.000000,0.00000,10624.260830,5b8e595b8cebb006e39a3d289e465a5718996785008cbb...,bd1b475ded03bdb3ab78de2f74782ad855c3c4b085d167...,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
f12e3d3e772d0ed5c7d02a22d9a9d7332b4af151cf7b3ba222a72d8f3511d552,2267c5d8b45462d96645afaa6a58323ae42552fe1decfe76e5277fb1c5685cfd,202203,0.362856,58.0,18.0,40.0,12.4,113.49,1.0,grupo_8,bajo,...,0.0,0.000000,0.000000,0.00000,10.250733,5b8e595b8cebb006e39a3d289e465a5718996785008cbb...,bd1b475ded03bdb3ab78de2f74782ad855c3c4b085d167...,1,0,0
7a18ff4eacdc583745e44ce978868ce1cbeae06aaa2b38c50278fa4a9b1641bb,0df6b75991158b581d702c968668b887acdd5a925696e70e259a924145815720,202203,0.362856,51.0,4.0,46.0,NaN,NaN,NaN,grupo_1|grupo_10|grupo_8|grupo_9|grupo_11,medio,...,0.0,0.559992,6.036156,44.49746,2874.128489,5b8e595b8cebb006e39a3d289e465a5718996785008cbb...,bd1b475ded03bdb3ab78de2f74782ad855c3c4b085d167...,1,0,0
3ce5571a754decadcd896a7ec49e1f573495a036c2e58c0fce845fc723904c61,6b1dea2e0452c374df981eb5a4e678bf4995684d03e0215acbfd3890564bd069,202202,0.362856,79.0,70.0,9.0,NaN,NaN,NaN,grupo_6|grupo_8,alto,...,0.0,0.000000,0.000000,0.00000,19942.603811,582e9885b25fb75af93c22f82ec43141b0adfa033e8389...,253cacec3a3cd7996e46e636aacf2a22a13a95aa2bb25e...,1,0,0
